# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [4]:
SEED = 100

## Solution obtained with Ant Colony Optimization

In [5]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-CRR'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

aco_history = []
solutions = {}
for filename in files:
    if filename.endswith('.xlsx'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

    def run_experiment(experiment: int):
        aco_params = dict(
            iterations=50,
            num_ants=len(terminals),
            evaporation_rate=0.5,
            alpha=1.0,
            beta=3.0,
            # beta_evaporation_rate=0.2,
            initial_pheromone=0.1,
            pheromone_amount=2.0,
            pheromone_deposit_strategy='best_route',
            pheromone_initialization_strategy='same_value',
            choose_best=0.2,
            log_level='info',
            early_stopping=10,
            normalize_distance_prize=False,
            allow_edge_perturbation=False,
            ant_max_moves=1000,
            seed=SEED * experiment
        )
        solver = AntColony(
            graph=stp_preprocessed.graph,
            terminals=stp_preprocessed.terminals,
            **aco_params
        )
        steiner_tree, steiner_cost = solver.solve()

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": steiner_cost,
            "duration": solver._duration
        }
        history.update(aco_params)
        return history, solver

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    aco_history.extend([result[0] for result in experiments_results])
    solutions[filename] = [result[1] for result in experiments_results]

Reading: ../data/instances/benchmark/PCSPG-CRR/C01-A.stp


2022-02-02 13:31:58,259 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-02 13:31:58,261 - [aco.py:302] - MainThread - INFO - Best Iteration: 7 - Best Cost: 92.0
2022-02-02 13:32:00,056 - [aco.py:295] - MainThread - INFO - Early Stopping: 13
2022-02-02 13:32:00,059 - [aco.py:302] - MainThread - INFO - Best Iteration: 4 - Best Cost: 84.0
2022-02-02 13:32:00,484 - [aco.py:295] - MainThread - INFO - Early Stopping: 15
2022-02-02 13:32:00,486 - [aco.py:302] - MainThread - INFO - Best Iteration: 9 - Best Cost: 91.0
2022-02-02 13:32:01,178 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-02 13:32:01,180 - [aco.py:302] - MainThread - INFO - Best Iteration: 10 - Best Cost: 78.0
2022-02-02 13:32:02,977 - [aco.py:295] - MainThread - INFO - Early Stopping: 14
2022-02-02 13:32:02,980 - [aco.py:302] - MainThread - INFO - Best Iteration: 13 - Best Cost: 77.0


Reading: ../data/instances/benchmark/PCSPG-CRR/C05-A.stp


In [ ]:
experiment = 4
G = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment].graph
steiner_tree = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment].steiner_tree

In [ ]:
sum(list(nx.get_node_attributes(G, 'prize').values()))

439.0

In [ ]:
steiner_tree.nodes

NodeView((76, 232, 78))

In [ ]:
G.nodes[78]

{'prize': 6.0, 'terminal': 6.0}

In [ ]:
import pandas as pd

df_score_aco = pd.DataFrame.from_dict(aco_history)
df_score_aco.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'ACO.csv'))

In [ ]:
df_score_aco

,filename,experiment,num_nodes,num_edges,num_nodes_after_preprocessing,num_edges_after_preprocessing,terminals,steiner_cost,duration
0,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,1,500,625,311,436,5,88.0,0.663081
1,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,2,500,625,311,436,5,88.0,0.600559
2,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,3,500,625,311,436,5,88.0,0.644161
3,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,4,500,625,311,436,5,88.0,0.678591
4,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,5,500,625,311,436,5,88.0,0.678483
5,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,6,500,625,311,436,5,88.0,0.688529
6,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,7,500,625,311,436,5,88.0,0.672962
7,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,8,500,625,311,436,5,88.0,0.671533
8,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,9,500,625,311,436,5,88.0,0.662299
9,../data/instances/benchmark/PCSPG-CRR/C01-A.stp,10,500,625,311,436,5,88.0,0.676295
